In [17]:
from app.llm_api.zhipu_api import Zhipu

In [18]:
zhipu = Zhipu()

In [19]:
response = zhipu.synchronous_chat([
    {
        "role": "user",
        "content": "1+1=?"
    }
])

In [20]:
response

CompletionMessage(content='1+1=2。', role='assistant', tool_calls=None)

In [21]:
response.content

'1+1=2。'

In [22]:

from langchain.vectorstores import DocArrayInMemorySearch

In [23]:
from langchain.document_loaders import TextLoader

In [24]:
# file = '悟空传.txt'
# file = '狂人日记.txt'

In [25]:
# files = ['悟空传.txt', '狂人日记.txt']
# loaders = []
# for file in files:
#     loader = TextLoader(file_path=file)
#     loaders.append(loader)

In [ ]:
files = ['悟空传.txt', '狂人日记.txt']
documents = []
for file in files:
    loader = TextLoader(file_path=file)
    docs = loader.load()
    for doc in docs:
        doc.metadata['source'] = file.split('.')[0]
    documents.extend(docs)
documents

In [27]:
# loader = TextLoader(file_path=file)

In [28]:
from langchain.indexes import VectorstoreIndexCreator

In [29]:
from langchain_community.embeddings import ZhipuAIEmbeddings
from app.config.config import settings

embeddings = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=settings.zhipu_api_key,
)

# index = VectorstoreIndexCreator(
#     vectorstore_cls=DocArrayInMemorySearch, 
#     embedding=embeddings
# ).from_loaders(loaders)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_documents(documents)

In [30]:
# query = '孙悟空为什么不杀了那个妖怪？'
# query = '历史是什么？'
query = '总结一下'
for doc in documents:
    print(doc.metadata)

{'source': '悟空传'}
{'source': '狂人日记'}


In [31]:
# 创建检索器，应用文件名过滤器
retriever = index.vectorstore.as_retriever(search_type="similarity", search_kwargs={"source": "悟空传"})
retriever.invoke(query, filter={"source": "悟空传"})


[Document(metadata={'source': '狂人日记'}, page_content='四\n早上，我静坐了一会。陈老五送进饭来，一碗菜，一碗蒸鱼；这鱼的眼睛，白而且硬，张着嘴，同那一伙想吃人的人一样。吃了几筷，滑溜溜的不知是鱼是人，便把他兜肚连肠的吐出。\n我说“老五，对大哥说，我闷得慌，想到园里走走。”老五不答应，走了，停一会，可就来开了门。\n我也不动，研究他们如何摆布我；知道他们一定不肯放松。果然！我大哥引了一个老头子，慢慢走来；他满眼凶光，怕我看出，只是低头向着地，从眼镜横边暗暗看我。大哥说：“今天你仿佛很好。”我说：“是的。”大哥说：“今天请何先生来，给你诊一诊。”我说：“可以！”其实我岂不知道这老头子是刽子手扮的！无非借了看脉这名目，揣一揣肥瘠：因这功劳，也分一片肉吃。我也不怕；虽然不吃人，胆子却比他们还壮。伸出两个拳头，看他如何下手。老头子坐着，闭了眼睛，摸了好一会，呆了好一会；便张开他鬼眼睛说：“不要乱想。静静的养几天，就好了。”\n不要乱想，静静的养！养肥了，他们是自然可以多吃；我有什么好处，怎么会“好了”？他们这群人，又想吃人，又是鬼鬼祟祟，想法子遮掩，不敢直捷下手，真要令我笑死，我忍不住，便放声大笑起来，十分快活。自己晓得这笑声里面，有的是义勇和正气。老头子和大哥，都失了色，被我这勇气正气镇压住了。\n但是我有勇气，他们便越想吃我，沾光一点这勇气。老头子跨出门，走不多远，便低声对大哥说道：“赶紧吃罢！”大哥点点头。原来也有你！这一件大发见，虽似意外，也在意中：合伙吃我的人，便是我的\n哥哥！\n吃人的是我哥哥！\n我是吃人的人的兄弟！\n我自己被人吃了，可仍然是吃人的人的兄弟！\n\n五\n这几天是退一步想：假使那老头子不是刽子手扮的，真是医生，也仍然是吃人的人。他们的祖师李时珍做的“本草什么”上，明明写着人肉可以煎吃；他还能说自己不吃人么？\n至于我家大哥，也毫不冤枉他。他对我讲书的时候，亲口说过可以“易子而食”；又一回偶然议论起一个不好的人，他便说不但该杀，还当“食肉寝皮”。我那时年纪还小，心跳了好半天。前天狼子村佃户来说吃心肝的事，他也毫不奇怪，不住的点头。可见心思是同从前一样狠。既然可以“易子而食”，便什么都易得，什么人都吃得。我从前单听他讲道理，也胡涂过去；现在晓得他讲道理的时候，不但唇边还抹着人油，而且心里满装着

In [32]:
# from langchain_openai import ChatOpenAI
# 
# llm = ChatOpenAI(
#     temperature=0,
#     model="glm-4",
#     openai_api_key=settings.zhipu_api_key,
#     openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
# )
# # 创建 RetrievalQA chain
# from langchain.chains import RetrievalQA
# qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
# response = qa_chain.run(query)
# response
# response = index.query(query, llm)
# response = index.query(query, llm, filters={"file_id": '悟空传'})